#Imports

In [1]:
#Instalaciones para descargar las imágenes al entorno
!pip install gdown --upgrade --no-cache-dir
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [9]:
#Imports
from google.colab import auth
from googleapiclient.discovery import build
import pandas as pd
import gdown
import io
import os
from googleapiclient.http import MediaIoBaseDownload

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import shutil
import random
from PIL import Image
from torchvision import transforms

# Crear dataset

In [3]:
#Preparar el entorno para descargar los archivos
auth.authenticate_user()
drive_service = build('drive', 'v3')

#Links de acceso público a las carpetas con las imágenes
folder_ia = '14xM8jjGl7QPMDUkti6iUl4M1m6qcs4L1'
folder_real = '1ruvbcczrUZ2W6rmmC7TvLUQxtOD_Cg3J'

#Función para descargar las imágenes al entorno
def download_files_from_folder(folder_id, output_folder):
  query = f"'{folder_id}' in parents"
  results = drive_service.files().list(q=query, fields='nextPageToken, files(id, name)').execute()
  items = results.get('files', [])

  if not items:
    print('No hay archivos en la carpeta.')
    return

  os.makedirs(output_folder, exist_ok=True)

  for item in items:
    file_id = item['id']
    file_name = item['name']
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
      status, done = downloader.next_chunk()
    with open(os.path.join(output_folder, file_name), 'wb') as f:
      f.write(fh.getbuffer())

# Call the function to download the files
download_files_from_folder(folder_ia, 'fotos_ia')
print(f"Carpeta fotos_ia completada.")
download_files_from_folder(folder_real, 'fotos_real')
print(f"Carpeta fotos_real completada.")

Carpeta fotos_ia completada.
Carpeta fotos_real completada.


In [20]:
###Resize las imagenes para que tengan las mismas dimensiones

carpetas = {
    'real': '/content/fotos_real',
    'ia': '/content/fotos_ia'
}
os.makedirs('/content/resize', exist_ok=True)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.ToPILImage()
])


for nombre, path in carpetas.items():
    os.makedirs(f"resize/{nombre}", exist_ok=True)

    for nombre_imagen in os.listdir(path):
        ruta_img = os.path.join(path, nombre_imagen)
        try:
            with Image.open(ruta_img) as img:
                img = img.convert('RGB')
                img_transformada = transform(img)
                img_transformada.save(os.path.join(f"resize/{nombre}", nombre_imagen))

        except Exception as e:
            print(f"Error procesando {ruta_img}: {e}")


In [22]:
#Crear dataset final
carpetas = {
    'real': '/content/resize/real',
    'ia': '/content/resize/ia'
}

#Crear carpetas de sets --> datasets>[train>[real, ia], test>[real, ia], val>[real, ia]]
os.makedirs(f"dataset", exist_ok=True)
for sets in ['train', 'test', 'val']:
  for origen in ['real', 'ia']:
    os.makedirs(f"dataset/{sets}/{origen}", exist_ok=True)

#Guardar las fotos en las imagenes en su respectiva carpeta
for nombre, path in carpetas.items():
  imagenes = os.listdir(path)

  #Las imágenes reales están ordenadas por edad, queremos evitar que un set contenga solo un rango de
  random.shuffle(imagenes)

  train = imagenes[:int(len(imagenes)*0.7)] #70% de contenido para el training set
  val = imagenes[:int(len(imagenes)*0.15)] #15% de contenido para el validation set
  test = imagenes[(int(len(imagenes)*0.7) + int(len(imagenes)*0.15)):] #15% de contenido para el test set

  for foto in train:
    shutil.copy(os.path.join(path, foto), f"dataset/train/{nombre}/{foto}")

  for foto in val:
    shutil.copy(os.path.join(path, foto), f"dataset/val/{nombre}/{foto}")

  for foto in test:
    shutil.copy(os.path.join(path, foto), f"dataset/test/{nombre}/{foto}")

# Debug

In [19]:
###DEBUG --> borrar los contenidos de una carpeta###
import os
import shutil

carpeta = "/content/dataset"

for archivo in os.listdir(carpeta):
    ruta_completa = os.path.join(carpeta, archivo)
    if os.path.isfile(ruta_completa) or os.path.islink(ruta_completa):
        os.unlink(ruta_completa)
    elif os.path.isdir(ruta_completa):
        shutil.rmtree(ruta_completa)
